In [ ]:
!git clone https://github.com/google-research/google-research.git

In [ ]:
!cd google-research/goemotions

In [ ]:
!ls

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report


from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


In [ ]:
!pip install scikit-plot
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
# Paths to the csv files
train_path = 'google-research/goemotions/data/train.tsv'
test_path = 'google-research/goemotions/data/test.tsv'
dev_path = 'google-research/goemotions/data/dev.tsv'

# Load the data
train_df = pd.read_csv(train_path, sep='\t', header=None, names=['text', 'emotion', 'id'])
test_df = pd.read_csv(test_path, sep='\t', header=None, names=['text', 'emotion', 'id'])
dev_df = pd.read_csv(dev_path, sep='\t', header=None, names=['text', 'emotion', 'id'])

# Combine train, test, and dev data into a single DataFrame
data = pd.concat([train_df, test_df, dev_df], ignore_index=True)
text_column_index = data.columns[0]

# Inspect the dataset
print(data.head())

# Preprocess (basic example)
data['text'] = data[text_column_index]
data['text'] = data['text'].str.lower()
# data['emotion'] = data.columns[1]


def preprocess_text(text):
    # Convert to lowercase and remove special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())

    # Tokenize
    tokens = text.split()

    # Remove stopwords and lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return ' '.join(tokens)

data['processed_text'] = data['text'].apply(preprocess_text)
# Split the data
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['emotion'], test_size=0.2, random_state=42)


In [ ]:
data['emotion'].value_counts(50)

In [ ]:
data = data.iloc[:50]

In [ ]:
sns.countplot(x='emotion', data=data)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Define the classifiers to be tested
classifiers = {
    # 'LinearSVC': LinearSVC(),
    # 'MultinomialNB': MultinomialNB(),
    # 'RandomForestClassifier': RandomForestClassifier(),
    'LogisticRegression': LogisticRegression(max_iter=200)
}

# Define the parameter grids for each classifier
param_grids = {
    'LinearSVC': {
        'tfidf__max_features': [3000, 5000, 10000],
        'tfidf__ngram_range': [(1, 1), (1, 2)],
        'classifier__C': [0.1, 1, 10]
    },
    # 'MultinomialNB': {
    #     'tfidf__max_features': [3000, 5000, 10000],
    #     'tfidf__ngram_range': [(1, 1), (1, 2)],
    #     'classifier__alpha': [0.1, 1, 10]
    # },
    # 'RandomForestClassifier': {
    #     'tfidf__max_features': [3000, 5000, 10000],
    #     'tfidf__ngram_range': [(1, 1), (1, 2)],
    #     'classifier__n_estimators': [100, 200],
    #     'classifier__max_depth': [None, 10, 20]
    # }
    'LogisticRegression': {
        'tfidf__max_features': [3000, 5000, 10000],
        'tfidf__ngram_range': [(1, 1), (1, 2)],
        'classifier__C': [0.1, 1, 10]
    }
}

# Loop through each classifier
for name, classifier in classifiers.items():
    print(f"Running GridSearchCV for {name}...")

    # Create a pipeline
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('classifier', classifier)
    ])

    # Perform cross-validation
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, error_score='raise')
    print(f"{name} Cross-validation scores: {cv_scores}")
    print(f"{name} Mean CV score: {np.mean(cv_scores):.3f} (+/- {np.std(cv_scores) * 2:.3f})")

    # Grid search for hyperparameter tuning
    grid_search = GridSearchCV(pipeline, param_grids[name], cv=5, n_jobs=-1, verbose=1)
    grid_search.fit(X_train, y_train)

    print(f"{name} Best parameters: {grid_search.best_params_}")
    print(f"{name} Best cross-validation score: {grid_search.best_score_:.3f}")

    # Evaluate on test set
    y_pred = grid_search.predict(X_test)
    print(f"Classification Report for {name}:")
    print(classification_report(y_test, y_pred))
    print("\n" + "="*60 + "\n")



In [ ]:
DO NOT
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# smote = SMOTE(random_state=42)
# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(data['text'], data['emotion'], test_size=0.2, random_state=42)
# vectorizer = TfidfVectorizer()
# X_train_vec = vectorizer.fit_transform(X_train)
# X_test_vec = vectorizer.transform(X_test)

# X_train_resampled, y_train_resampled = smote.fit_resample(X_train.values.reshape(-1, 1), y_train )
y_pred = grid_search.predict(X_test)
cm = confusion_matrix(y_true=y_test, y_pred=y_pred, normalize='true')

print("Confusion Matrix:")
print(cm)

# Plot the confusion matrix
plt.figure(figsize=(15, 12))
sns.heatmap(cm, annot=True, fmt='.2f', cmap='Blues', xticklabels=data['emotion'].unique(), yticklabels=data['emotion'].unique(), norm=LogNorm())
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Normalized Confusion Matrix (Log Scale)')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from matplotlib.colors import LogNorm

y_pred = grid_search.predict(X_test)
cm = confusion_matrix(y_true=y_test, y_pred=y_pred, normalize='true')

print("Confusion Matrix:")
print(cm)

# Manually select a subset of numerical labels
selected_labels = [0, 17, 2, 26, 14, 25, 18, 11, 4, 27, 5,6,7,8,9,1,11,15,16,18,19]  # Replace these with the labels you want to focus on

# Map the selected labels to their corresponding emotion names
selected_emotion_names = [ data['emotion'].iloc[i] for i in selected_labels ]

# Filter the confusion matrix and labels accordingly
cm_filtered = cm[np.ix_(selected_labels, selected_labels)]

plt.figure(figsize=(12, 10))

# Plot the confusion matrix with adjustments
sns.heatmap(cm_filtered, annot=True, fmt='.2f', cmap='Blues',
            xticklabels=selected_emotion_names, yticklabels=selected_emotion_names,
            norm=LogNorm(), cbar_kws={'label': 'Log scale'})

plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.yticks(rotation=0)  # Keep y-axis labels horizontal

plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Normalized Confusion Matrix (Log Scale)')

plt.tight_layout()  # Adjust the layout to make room for rotated labels
plt.show()


In [ ]:
# pipeline = Pipeline([
#     ('tfidf', TfidfVectorizer(max_features=5000)),
#     ('classifier', LinearSVC()),
    # ('classifier', RandomForestClassifier()),
    # ('classifier', MultinomialNB()),
    # ('classifier', LogisticRegression()),
    # ('classifier', SVC()),
# ])


In [ ]:
# Create a pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000)),
    ('classifier', LinearSVC())
])
print(y_train.value_counts())
# Perform cross-validation
cv_scores = cross_val_score(pipeline, dev_df, train_df, cv=5, error_score='raise')
print(f"Cross-validation scores: {cv_scores}")
print(f"Mean CV score: {np.mean(cv_scores):.3f} (+/- {np.std(cv_scores) * 2:.3f})")

# Grid search for hyperparameter tuning
param_grid = {
    'tfidf__max_features': [3000, 5000, 10000],
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'classifier__C': [0.1, 1, 10]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_:.3f}")

# Evaluate on test set
y_pred = grid_search.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
# Example prediction
new_text = ["You look so sad"]
new_text_processed = [preprocess_text(text) for text in new_text]
prediction = grid_search.predict(new_text_processed)
print(f"Predicted emotion: {prediction}")

In [ ]:
# Save the model using joblib
import joblib
model_filename = 'project_emotion_detection_model.pkl'
joblib.dump(grid_search.best_estimator_, model_filename)
